In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df = pd.read_csv("SignList_ClassId_TR_EN.csv", encoding='unicode_escape')

df.head()



,ClassId,TR,EN
0,0,abla,sister
1,1,acele,hurry
2,2,acýkmak,hungry
3,3,afiyet_olsun,enjoy_your_meal
4,4,aðabey,brother
